In [1]:
import folium

from openrouteservice import client
from folium.plugins import MousePosition

In [2]:
api_key = 'your_key_here'
ors = client.Client(key=api_key)

# Set up folium map
map2 = folium.Map(tiles='Stamen Toner',
                  location=([51.463010, -0.113592]),
                  zoom_start=12
                 )

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

map2.add_child(mouse_position)

# Set up the apartment dictionary with real coordinates
apartments = {'first': {'location': [-0.109335, 51.470446]},
              'second': {'location': [-0.08291, 51.49312]},
              'third': {'location': [-0.06288, 51.47645]}
              }

params_iso = {'profile': 'foot-walking',
              'range': [900],  # 900/60 = 15 minutes
              'attributes': ['total_pop'] 
              }

for name, apt in apartments.items():
    params_iso['locations'] = [apt['location']]  # Add apartment coords to request parameters
    apt['iso'] = ors.isochrones(**params_iso)  
    folium.features.GeoJson(apt['iso']).add_to(map2)  # Add GeoJson to map

    folium.map.Marker(list(reversed(apt['location'])),
                      icon = folium.Icon(color = 'beige',
                                         icon_color = '#7911EF',
                                         icon = 'home',
                                         prefix = 'fa',
                                         ),
                      popup = name,
                      ).add_to(map2)  # Add apartment locations to map
    
map2

In [6]:
# Common request parameters
params_poi = {'request': 'pois',
              'sortby': 'distance'}

# POI categories according to
categories_poi = {'bar': [561],
                  'supermarket': [518],
                  'bank': [192]}

for name, apt in apartments.items():
    apt['categories'] = dict()  # Store in pois dict for easier retrieval
    params_poi['geojson'] = apt['iso']['features'][0]['geometry']
    print("\n{} apartment".format(name))

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        apt['categories'][typ] = dict()
        apt['categories'][typ]['geojson'] = ors.places(**params_poi)['features']  # Actual POI request
        print(f"\t{typ}: {len(apt['categories'][typ]['geojson'])}")
    


first apartment
	bar: 11
	supermarket: 6
	bank: 8

second apartment
	bar: 14
	supermarket: 12
	bank: 6

third apartment
	bar: 8
	supermarket: 13
	bank: 7


In [8]:
# Set up common request parameters
params_route = {'profile': 'foot-walking',
                'format_out': 'geojson',
                'geometry': 'true',
                'format': 'geojson',
                'instructions': 'false',
                }

# Set up dict for font-awesome
style_dict = {'bar': 'beer',
              'supermarket': 'shopping-cart',
              'bank': 'money'
              }

# Store all routes from all apartments to POIs
for apt in apartments.values():
    for cat, pois in apt['categories'].items():
        pois['durations'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']

            # Perform actual request
            params_route['coordinates'] = [apt['location'],
                                           poi_coords
                                           ]
            json_route = ors.directions(**params_route)

            folium.features.GeoJson(json_route).add_to(map2)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='beige',
                                               icon_color='#FC12AE',
                                               icon=style_dict[cat],
                                               prefix='fa'
                                               )
                              ).add_to(map2)

            poi_duration = json_route['features'][0]['properties']['summary']['duration']
            pois['durations'].append(poi_duration)  # Record durations of routes

map2

In [8]:
# Sum up the closest POIs to each apartment
for name, apt in apartments.items():
    apt['shortest_sum'] = sum([min(cat['durations']) for cat in apt['categories'].values()])
    print(f"{name} apartment: {round(apt['shortest_sum'] / 60, 1)} min"
          )

first apartment: 21.6 min
second apartment: 26.9 min
third apartment: 22.3 min
